## Solutions Actions Quiz

<img src="../img/robotignite_logo_text.png"/>

For this Quiz, we will assume that our package is called **actions_quiz**, our launch file is called **action_custom_msg.launch**, and our Python file is called **action_custom_msg.py**.

So, for the Quiz, we will have to create a custom action message that will defines the movement of the drone by using a string, that can be **UP** or **DOWN**. As feedback, it will also return a string indicating which action is taking place at the moment. As a result, it will return nothing.

For that, the first you will have to do is to create a folder called **action** inside the **actions_quiz** package.

In [ ]:
roscd; cd ..; cd src;
catkin_create_pkg actions_quiz;
roscd actions_quiz;
mkdir action;

Then, isnide this **action** folder, you will have to create a file called **CustomActionMsg.action**, with the following content inside it:

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Message File: CustomActionMsg.action** </p>

In [ ]:
string goal
---
---
string feedback

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Message File: CustomActionMsg.action** </p>

As you can see, we define both the goal and the feedback as strings, while we leave the result in blank.

Then, you will also have to modify the **CMakeLists.txt** and **package.xml** files, as it is described in the Actions Notebooks. If you are lost and don't know how to proceed, below you can check working examples of this files:

<p style="background:#3B8F10;color:white;" id="prg-2-1">**CMakeLists.txt** </p>

In [ ]:
cmake_minimum_required(VERSION 2.8.3)
project(actions_quiz)

## Find catkin macros and libraries
## if COMPONENTS list like find_package(catkin REQUIRED COMPONENTS xyz)
## is used, also find other catkin packages
find_package(catkin REQUIRED COMPONENTS
  std_msgs 
  actionlib_msgs
)

## Generate actions in the 'action' folder
add_action_files(
   FILES
   CustomActionMsg.action
 )

## Generate added messages and services with any dependencies listed here
generate_messages(
   DEPENDENCIES
   std_msgs actionlib_msgs
 )

catkin_package(
 CATKIN_DEPENDS rospy
)

## Specify additional locations of header files
## Your package locations should be listed before other locations
# include_directories(include)
include_directories(
  ${catkin_INCLUDE_DIRS}
)

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END CMakeLists.txt** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**package.xml** </p>

In [ ]:
<?xml version="1.0"?>
<package format="2">
  <name>actions_quiz</name>
  <version>0.0.0</version>
  <description>The actions_quiz package</description>
  <maintainer email="user@todo.todo">user</maintainer>
  <license>TODO</license>

  <buildtool_depend>catkin</buildtool_depend>
  <build_depend>actionlib</build_depend>
  <build_depend>actionlib_msgs</build_depend>
  <build_depend>rospy</build_depend>
  <build_depend>std_msgs</build_depend>
  <build_export_depend>actionlib</build_export_depend>
  <build_export_depend>actionlib_msgs</build_export_depend>
  <build_export_depend>rospy</build_export_depend>
  <exec_depend>actionlib</exec_depend>
  <exec_depend>actionlib_msgs</exec_depend>
  <exec_depend>rospy</exec_depend>

  <export>
  </export>
</package>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END package.xml** </p>

Once all of these is done, you will need to compile your package and source **ALL the webshells** that you are going to use so that ROS can find the new Messages.

In [ ]:
roscd;
cd ..;
catkin_make;
source devel/setup.bash;

And finally, check if you can find your new message:

In [ ]:
rosmsg list | grep CustomActionMsg

You should see something like this:

In [ ]:
actions_quiz/CustomActionMsgAction
actions_quiz/CustomActionMsgActionFeedback
actions_quiz/CustomActionMsgActionGoal
actions_quiz/CustomActionMsgActionResult
actions_quiz/CustomActionMsgFeedback
actions_quiz/CustomActionMsgGoal
actions_quiz/CustomActionMsgResult

Once this is done and working, you can proceed to create the Action Server that will use this new message. Below you can check both the launch file and the Python file:

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Launch File: action_custom_msg.launch** </p>

In [ ]:
<launch>
    <node pkg="actions_quiz" type="action_custom_msg.py" name="action_custom_msg" output="screen" />
</launch>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Launch File: action_custom_msg.launch** </p>

<p style="background:#3B8F10;color:white;" id="prg-2-1">**Python File: action_custom_msg.py** </p>

In [ ]:
#! /usr/bin/env python
import rospy
import time
import actionlib

from actions_quiz.msg import CustomActionMsgFeedback, CustomActionMsgResult, CustomActionMsgAction
from geometry_msgs.msg import Twist
from std_msgs.msg import Empty

class CustomActionMsgClass(object):
    
  # create messages that are used to publish feedback/result
  _feedback = CustomActionMsgFeedback()
  _result   = CustomActionMsgResult()

  def __init__(self):
    # creates the action server
    self._as = actionlib.SimpleActionServer("action_custom_msg_as", CustomActionMsgAction, self.goal_callback, False)
    self._as.start()

    
  def goal_callback(self, goal):
    # this callback is called when the action server is called.
    
    # helper variables
    success = True
    r = rospy.Rate(1)
    
    # define the different publishers and messages that will be used
    self._pub_takeoff = rospy.Publisher('/drone/takeoff', Empty, queue_size=1)
    self._takeoff_msg = Empty()
    self._pub_land = rospy.Publisher('/drone/land', Empty, queue_size=1)
    self._land_msg = Empty()
    
    # Get the goal word: UP or DOWN
    takeoff_or_land = goal.goal
    
    i = 0
    for i in xrange(0, 4):
    
      # check that preempt (cancelation) has not been requested by the action client
      if self._as.is_preempt_requested():
        rospy.loginfo('The goal has been cancelled/preempted')
        # the following line, sets the client in preempted state (goal cancelled)
        self._as.set_preempted()
        success = False
        break
    
      # Logic that makes the robot move UP or DOWN
      if takeoff_or_land == 'TAKEOFF':
        
        self._pub_takeoff.publish(self._takeoff_msg)
        self._feedback.feedback = 'Taking Off...'
        self._as.publish_feedback(self._feedback)
    
      if takeoff_or_land == 'LAND':
        
        self._pub_land.publish(self._land_msg)
        self._feedback.feedback = 'Landing...'
        self._as.publish_feedback(self._feedback)
    
      # the sequence is computed at 1 Hz frequency
      r.sleep()
    
    # at this point, either the goal has been achieved (success==true)
    # or the client preempted the goal (success==false)
    # If success, then we publish the final result
    # If not success, we do not publish anything in the result
    if success:
      self._result = Empty()
      self._as.set_succeeded(self._result)
      
      
if __name__ == '__main__':
  rospy.init_node('action_custom_msg')
  CustomActionMsgClass()
  rospy.spin()

<p style="background:#3B8F10;color:white;" id="prg-2-1">**END Python File: action_custom_msg.py** </p>

As you can see, in the above code we are using the **/drone/takeoff** and **/drone/land** topics in order to move UP or DOWN, since they make the drone move aproximately 1 meter. You could also use the **/cmd_vel** topic, but that's up to you.

In order to test this code, you have to first launch the Action Server (which is defined in the above Python file). You can do that by using the following command:

In [ ]:
roslaunch actions_quiz action_custom_msg.launch

Now, if you execute the following command:

In [ ]:
rostopic list | grep action_custom_msg

You will see something like this:

<img src="../img/action_custom_msg_as.png" width="600" />

This means you Action Server is up and ready to receive a goal.

So now, let's publish a goal to this Action Server. You can do this by two methods (as you saw in the Part 1 of the Actions notebooks):

* Creating an Action Client
* Publishing a goal directly through the action topics

For this case, let's use the 2nd method. You should then publish a message to the **action_custom_msg_as/goal** topic, just like this:

In [ ]:
rostopic pub /action_custom_msg_as/goal actions_quiz/CustomActionMsgActionGoal "header:
  seq: 0
  stamp:
    secs: 0
    nsecs: 0
  frame_id: ''
goal_id:
  stamp:
    secs: 0
    nsecs: 0
  id: ''
goal:
  goal: 'TAKEOFF'"

You will see the drone doing something like this:

<img src="../img/up_or_down1.gif" width="600" />

<img src="../img/up_or_down2.gif" width="600" />

Finally, you can check the **feedback** and **result** topics of the Action, to check if they're publishing the desired values.

In [ ]:
rostopic echo /action_custom_msg_as/feedback

<img src="../img/actions_quiz_takeoff.png" width="600" />

<img src="../img/actions_quiz_landing.png" width="600" />

In this case, there's no result to be shown.